In [1]:
import numpy as np
import pandas as pd
import pyodbc 
import mysql.connector
from datetime import datetime

In [2]:
date = '2021-09-01'
ASM = 'Ánh'

selected_date = "'{}%'".format(date)
if(ASM == ''):  
    selected_ASM = "Á"
elif(ASM == ''):
    selected_ASM = "T"
else:
    selected_ASM = "H"

In [92]:
final_df = pd.DataFrame(columns = [])

In [ ]:
date = '2021-09-01'

for ASM in ["", "", ""]:
    selected_date = "'{}%'".format(date)
    if(ASM == ''):  
        selected_ASM = ""
    elif(ASM == ''):
        selected_ASM = ""
    else:
        selected_ASM = ""

    # Fetch all data with type=1 for subtraction
    connection = mysql.connector.connect(host='',
                                             database='',
                                             user='',
                                             password='')

    cursor = connection.cursor()
    mySql_Create_Table_Query = '''
    SELECT b.id, b.depotId, b.createdDateTime, saleId, discount, money, usedPointsMoney, d.ASM, d.Store
    FROM . as b

    LEFT JOIN . as d ON b.depotId=d.DepotId
    WHERE b.Type = 1 AND b.Mode = 2 AND d.ASM LIKE '{}%' AND b.createdDateTime LIKE {}
    '''.format(selected_ASM, selected_date)

    cursor.execute(mySql_Create_Table_Query)

    data = cursor.fetchall()
    data_df = pd.DataFrame(data, columns=cursor.column_names)

    # Save for analysis
    df = data_df

    df["mone"] = df["money"].apply(lambda x: float(x))
    df["usedPointsMoney"] = df["usedPointsMoney"].apply(lambda x: float(x))

    # Doanh Thu = DTLD - Tieu Diem
    df["Doanh Thu"] = df["money"] - df["usedPointsMoney"]

    sum_money = df.groupby("saleId")['Doanh Thu'].sum()

    # generate a string form to query order line items in database
    arr = ""

    arr = arr + "("
    for i in np.unique(df.id.values):
        arr = arr + str(i)
        arr = arr + ", "

    arr = arr[:-2]
    arr = arr + ")"

    # Query order details for each id
    connection = mysql.connector.connect(host='',
                                             database='',
                                             user='',
                                             password='')

    cursor = connection.cursor()
    mySql_Create_Table_Query = '''
    SELECT * FROM .
    WHERE billId IN {};
    '''.format(arr)

    print(mySql_Create_Table_Query)

    cursor.execute(mySql_Create_Table_Query)

    data = cursor.fetchall()
    product_df = pd.DataFrame(data, columns=cursor.column_names)


    product_df["code3"] = product_df["code"].apply(lambda x: x[:3])

    def func(x):
        if(x == "KID"):
            return "kid"
        elif(x == "APM" or x == "APN"):
            return "polo"
        else:
            return "sale_60%"

    product_df["marked"] = product_df.code3.apply(lambda x: func(x))

    product_df["price"] = product_df["price"].apply(lambda x: float(x))
    product_df["discount"] = product_df["discount"].apply(lambda x: float(x))
    product_df["money"] = product_df["money"].apply(lambda x: float(x))

    saleid_df = df[["saleId", "id"]]
    saleid_df.rename(columns={"id": "billId"}, inplace=True)

    total_df = product_df.merge(saleid_df, how="left", on="billId")

    marked_df = total_df.groupby(["saleId", "marked"])['money'].sum()
    marked_df2 = total_df.groupby(["saleId", "marked"])['money'].count()

    marked_df = pd.concat([marked_df, marked_df2], axis=1)
    marked_df.columns = ["money", "SL"]

    category_df = marked_df.unstack().reset_index()

    total_sale_df = pd.DataFrame(sum_money).reset_index()
    result = total_sale_df.merge(category_df, how="left", on="saleId")

    # Save doanh thu tra
    dtt_df = result

    # Doanh Thu Len Don - Doanh Thu Tra -> OKELA
    # Fetch all data with type=1 for subtraction

    connection = mysql.connector.connect(host='m',
                                             database='',
                                             user='',
                                             password='')

    cursor = connection.cursor()
    mySql_Create_Table_Query = '''
    SELECT b.id, b.depotId, b.createdDateTime, saleId, discount, money, usedPointsMoney, d.ASM, d.Store
    FROM . as b

    LEFT JOIN . as d ON b.depotId=d.DepotId
    WHERE b.Type = 2 AND b.Mode = 2 AND d. LIKE '{}%' AND b.createdDateTime LIKE {}
    '''.format(selected_ASM, selected_date)

    cursor.execute(mySql_Create_Table_Query)

    data = cursor.fetchall()
    data_df = pd.DataFrame(data, columns=cursor.column_names)

    # Save for analysis
    df = data_df

    df["money"] = df["money"].apply(lambda x: float(x))
    df["usedPointsMoney"] = df["usedPointsMoney"].apply(lambda x: float(x))

    # Doanh Thu = DTLD - Tieu Diem
    df["Doanh Thu"] = df["money"] - df["usedPointsMoney"]

    sum_money = df.groupby("saleId")['Doanh Thu'].sum()

    # generate a string form to query order line items in database
    arr = ""

    arr = arr + "("
    for i in np.unique(df.id.values):
        arr = arr + str(i)
        arr = arr + ", "

    arr = arr[:-2]
    arr = arr + ")"

    # Query order details for each id
    connection = mysql.connector.connect(host='',
                                             database='',
                                             user='',
                                             password='')

    cursor = connection.cursor()
    mySql_Create_Table_Query = '''
    SELECT * FROM .
    WHERE billId IN {};
    '''.format(arr)

    cursor.execute(mySql_Create_Table_Query)

    data = cursor.fetchall()
    product_df = pd.DataFrame(data, columns=cursor.column_names)


    product_df["code3"] = product_df["code"].apply(lambda x: x[:3])

    def func(x):
        if(x == "KID"):
            return "kid"
        elif(x == "APM" or x == "APN"):
            return "polo"
        else:
            return "sale_60%"

    product_df["marked"] = product_df.code3.apply(lambda x: func(x))

    product_df["price"] = product_df["price"].apply(lambda x: float(x))
    product_df["discount"] = product_df["discount"].apply(lambda x: float(x))
    product_df["money"] = product_df["money"].apply(lambda x: float(x))

    saleid_df = df[["saleId", "id"]]
    saleid_df.rename(columns={"id": "billId"}, inplace=True)

    total_df = product_df.merge(saleid_df, how="left", on="billId")

    marked_df = total_df.groupby(["saleId", "marked"])['money'].sum()
    marked_df2 = total_df.groupby(["saleId", "marked"])['money'].count()

    marked_df = pd.concat([marked_df, marked_df2], axis=1)
    marked_df.columns = ["money", "SL"]

    category_df = marked_df.unstack().reset_index()

    total_sale_df = pd.DataFrame(sum_money).reset_index()
    result = total_sale_df.merge(category_df, how="left", on="saleId")

    # result: dtld, dtt_df: dtt
    result.fillna(0.00, inplace=True)
    dtt_df.fillna(0.00, inplace=True)

    index_df = result[["saleId"]]

    dtt = index_df.merge(dtt_df, how="left", on="saleId").fillna(0.000)
    cols = result.columns.difference(['saleId'])
    result[cols] = result[cols].sub(dtt[cols])

    result["%polo"] = round(result[('money', 'polo')] / result["Doanh Thu"] * 100, 2)
    result["%sale_60%"] = round(result[('money', 'sale_60%')] / result["Doanh Thu"] * 100, 2)
    result["%kid"] = round(result[('money', 'kid')] / result["Doanh Thu"] * 100, 2)

    result["Date"] = date
    result["ASM"] = ASM

    final_df = final_df.append(result)
    
    print("--------------NEXT---INTERATION--------------")

In [69]:
final_df.to_pickle("1-2.pkl")

In [110]:
dff = pd.read_pickle("1-2.pkl")

In [111]:
dff["check_sum"] = dff[('money', 'kid')] + dff[('money', 'polo')] + dff[('money', 'sale_60%')]

In [66]:
final_df.to_pickle("1-9.pkl")

In [53]:
final_df = pd.read_pickle("1-9.pkl")

In [87]:
dsnv = pd.read_excel("dsnv.xlsx")

In [ ]:
dsnv = pd.read_excel("dsnv.xlsx")
dsnv["Cửa hàng"] = dsnv["Cửa hàng"].str.split('\n').str[-1]

dff.dropna(subset=["saleId"], inplace=True)
dff.replace({'saleId': {"": 0}}, inplace=True)
dff.saleId = dff.saleId.astype(str).astype(np.int64)

dff = pd.merge(dsnv, dff, how="right", right_on="saleId", left_on="ID")

result_rearranged = dff[['ID', 'Tên đăng nhập', 'Tên đầy đủ', 'Nhóm quyền', 'Điện thoại',
       'Email', 'Cửa hàng', 'Date', 'Doanh Thu',  ('money', 'kid'),
            ('money', 'polo'),  ('money', 'sale_60%'),          ('SL', 'kid'),
               ('SL', 'polo'),     ('SL', 'sale_60%'),                '%polo',
                  '%sale_60%',                 '%kid', 'ASM', "check_sum"]]

result_rearranged.to_excel("final.xlsx")
final = pd.read_excel("final.xlsx", header=None)
final.drop(columns=[0], inplace=True)

In [113]:
# ACCES GOOGLE SHEET
import gspread
from gspread_dataframe import set_with_dataframe

sheet_id = '12a-yoM2IR3DBa0YeudUiv2a_GmCgKfI7cZhgSIR2zOg'
gc = gspread.service_account(filename=r'GoogleAPICredentials.json') # <--Key
sh = gc.open_by_key(sheet_id) # <-- sheet id
worksheet = sh.worksheet('test2') #->gg sheet name

# APPEND DATA TO SHEET
your_dataframe = final
set_with_dataframe(worksheet, your_dataframe) #-> THIS EXPORTS YOUR DATAFRAME TO THE GOOGLE SHEET